In [14]:
# importando libs
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import pickle
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json
from prophet import plot
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [15]:
DAYS_PREDICT = 15

In [16]:
date_today = datetime.today().strftime("%Y-%m-%d")
data_init = "2014-01-01"

df_dai = yf.download("DAI-USD", data_init, date_today)

df_dai.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-11-22,0.997358,1.026548,0.972164,0.992598,0.992598,2104541
2019-11-23,0.992184,1.028747,0.988341,1.016282,1.016282,311283
2019-11-24,1.016288,1.035136,0.995609,1.002270,1.002270,255060
2019-11-25,1.001289,1.034708,0.984805,1.007021,1.007021,958777
2019-11-26,1.007208,1.031428,0.991767,1.006096,1.006096,217385


In [17]:
df_dai.reset_index(inplace=True)
df_dai.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
1025,2022-09-12,1.000274,1.000557,0.997372,0.999759,0.999759,469114833
1026,2022-09-13,0.999834,1.000297,0.996593,0.999750,0.999750,851331148
1027,2022-09-14,0.998508,1.000537,0.998172,1.000117,1.000117,698410149
1028,2022-09-15,1.000026,1.000665,0.998235,0.999694,0.999694,694549244
1029,2022-09-16,0.999802,1.000649,0.998466,1.000118,1.000118,442803639


In [18]:
df = df_dai[["Date", "Adj Close"]]
df.rename(columns= { "Date": "ds", "Adj Close": "y" }, inplace=True)

In [19]:
df.tail()

,ds,y
1025,2022-09-12,0.999759
1026,2022-09-13,0.999750
1027,2022-09-14,1.000117
1028,2022-09-15,0.999694
1029,2022-09-16,1.000118


In [20]:
assert df.isnull().sum().sum() == 0
assert df['ds'].isnull().sum() == 0
assert df['y'].isnull().sum() == 0

In [21]:
df.dtypes

ds    datetime64[ns]
y            float64
dtype: object

In [22]:
model = Prophet(seasonality_mode="multiplicative")
model.fit(df)

00:33:42 - cmdstanpy - INFO - Chain [1] start processing
00:33:43 - cmdstanpy - INFO - Chain [1] done processing


In [23]:
df_future = model.make_future_dataframe(periods=DAYS_PREDICT)
df_future.tail()

,ds
1040,2022-09-27
1041,2022-09-28
1042,2022-09-29
1043,2022-09-30
1044,2022-10-01


In [24]:
# predict 
predict = model.predict(df_future)
predict

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2019-11-22,1.007492,0.996685,1.012067,1.007492,1.007492,-0.002948,-0.002948,-0.002948,0.000394,0.000394,0.000394,-0.003343,-0.003343,-0.003343,0.0,0.0,0.0,1.004522
1,2019-11-23,1.007481,0.996585,1.012183,1.007481,1.007481,-0.003301,-0.003301,-0.003301,0.000048,0.000048,0.000048,-0.003349,-0.003349,-0.003349,0.0,0.0,0.0,1.004155
2,2019-11-24,1.007471,0.995290,1.010421,1.007471,1.007471,-0.004529,-0.004529,-0.004529,-0.001155,-0.001155,-0.001155,-0.003373,-0.003373,-0.003373,0.0,0.0,0.0,1.002908
3,2019-11-25,1.007460,0.997318,1.012436,1.007460,1.007460,-0.002932,-0.002932,-0.002932,0.000482,0.000482,0.000482,-0.003414,-0.003414,-0.003414,0.0,0.0,0.0,1.004506
4,2019-11-26,1.007449,0.996217,1.011641,1.007449,1.007449,-0.003924,-0.003924,-0.003924,-0.000453,-0.000453,-0.000453,-0.003471,-0.003471,-0.003471,0.0,0.0,0.0,1.003496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040,2022-09-27,0.992029,0.985474,1.000399,0.991997,0.992057,0.000744,0.000744,0.000744,-0.000453,-0.000453,-0.000453,0.001197,0.001197,0.001197,0.0,0.0,0.0,0.992767
1041,2022-09-28,0.991979,0.984625,1.000702,0.991930,0.992018,0.000566,0.000566,0.000566,-0.000173,-0.000173,-0.000173,0.000739,0.000739,0.000739,0.0,0.0,0.0,0.992540
1042,2022-09-29,0.991929,0.985412,1.001302,0.991858,0.991991,0.001183,0.001183,0.001183,0.000856,0.000856,0.000856,0.000328,0.000328,0.000328,0.0,0.0,0.0,0.993103
1043,2022-09-30,0.991879,0.983896,1.000520,0.991785,0.991959,0.000362,0.000362,0.000362,0.000394,0.000394,0.000394,-0.000033,-0.000033,-0.000033,0.0,0.0,0.0,0.992238


In [25]:
predict[["ds", "yhat_lower", "yhat_upper", "yhat"]].tail()

,ds,yhat_lower,yhat_upper,yhat
1040,2022-09-27,0.985474,1.000399,0.992767
1041,2022-09-28,0.984625,1.000702,0.992540
1042,2022-09-29,0.985412,1.001302,0.993103
1043,2022-09-30,0.983896,1.000520,0.992238
1044,2022-10-01,0.983458,0.998559,0.991539


In [26]:
# Salvar modelo
with open('model_dai_usd.pckl', 'wb') as fout:
    pickle.dump(model, fout)